Script for processing raw EN4 data prior to analysis.
See docstring of `Profile.process_en4()` for more specifics on what it does.

This script will just load modules, read in raw EN4 data, cut out a geographical box, call the processing routine and write the processed data to file.

You don't have to do this for each EN4 file individually if you don't want, you can read in multiple using `multiple = True` on the creation of the profile object. However, if analysing model data in parallel chunks, you may want to split up the processing into smaller files to make the analysis faster.

In [9]:
import sys

# IF USING A DEVELOPMENT BRANCH OF COAST, ADD THE REPOSITORY TO PATH:
# sys.path.append('<PATH_TO_COAST_REPO')
import coast
import pandas as pd
from datetime import datetime
from os import path

print("Modules loaded")

# File paths - input en4, output processed file and read config file
"""
fn_prof = "<PATH_TO_RAW_EN4_DATA_FILE(S)>"
fn_out = "<PATH_TO_OUTPUT_LOCATION_FOR_PROCESSED_PROFILES>"
fn_cfg_prof = "<PATH_TO_COAST_PROFILE_CONFIG_FILE>"
"""

fn_out = "./output.nc"
fn_prof = path.join('./example_files', "coast_example_en4_201008.nc")
fn_prof = path.join('/Users/jeff/Downloads/EN.4.2.2.profiles.g10.2022', "EN.4.2.2.f.profiles.g10.2022*.nc")
fn_cfg_prof = path.join('/Users/jeff/gitHub/COAsT/config', "example_en4_profiles.json")


# Some important settings, easier to get at here
longitude_bounds = [-15, 15]  # Geo box to cut out from data (match to model)
latitude_bounds = [40, 65]
multiple = True  # Reading multple files?

Modules loaded


In [17]:
fn_prof = "/Users/jeff/GitHub/COAsT/example_files/coast_example_en4_201008.nc"

In [10]:
# Some important settings, easier to get at here
longitude_bounds = [0, 360]  # Geo box to cut out from data (match to model)
latitude_bounds = [-15, 15]
multiple = True  # Reading multple files?

Create profile object containing data

In [18]:
profile = coast.Profile(config=fn_cfg_prof)
profile.read_en4(fn_prof, multiple=multiple)

/Users/jeff/gitHub/COAsT/config/example_en4_profiles.json


In [19]:
profile.dataset

<xarray.Dataset>
Dimensions:                   (id_dim: 1257, z_dim: 400)
Coordinates:
    latitude                  (id_dim) float64 dask.array<chunksize=(1257,), meta=np.ndarray>
    longitude                 (id_dim) float64 dask.array<chunksize=(1257,), meta=np.ndarray>
    time                      (id_dim) datetime64[ns] dask.array<chunksize=(1257,), meta=np.ndarray>
Dimensions without coordinates: id_dim, z_dim
Data variables:
    depth                     (id_dim, z_dim) float32 dask.array<chunksize=(1257, 400), meta=np.ndarray>
    potential_temperature     (id_dim, z_dim) float32 dask.array<chunksize=(1257, 400), meta=np.ndarray>
    temperature               (id_dim, z_dim) float32 dask.array<chunksize=(1257, 400), meta=np.ndarray>
    practical_salinity        (id_dim, z_dim) float32 dask.array<chunksize=(1257, 400), meta=np.ndarray>
    qc_potential_temperature  (id_dim, z_dim) |S1 dask.array<chunksize=(1257, 400), meta=np.ndarray>
    qc_practical_salinity     (id_dim, z_dim) |S1 dask.array<chunksize=(1257, 400), meta=np.ndarray>
    qc_depth                  (id_dim, z_dim) |S1 dask.array<chunksize=(1257, 400), meta=np.ndarray>
    qc_time                   (id_dim) |S1 dask.array<chunksize=(1257,), meta=np.ndarray>
    qc_flags_profiles         (id_dim) int32 dask.array<chunksize=(1257,), meta=np.ndarray>
    qc_flags_levels           (id_dim, z_dim) int32 dask.array<chunksize=(1257, 400), meta=np.ndarray>
Attributes:
    history:  Fri Apr 21 21:10:34 2017: /project/ukmo/rhel6/nco/bin/ncks -x -...
    NCO:      4.3.2

Get geographical indices to extract.

In [12]:
profile = profile.subset_indices_lonlat_box(longitude_bounds, latitude_bounds)

In [15]:
profile.quick_plot()

AttributeError: 'Profile' object has no attribute 'quick_plot'

Cut out a time slice of the data.

In [ ]:
profile = profile.time_slice(date0=datetime(2010, 1, 1), date1=datetime(2010, 1, 20))

Process the extracted data into new processed profile.

In [ ]:
processed_profile = profile.process_en4()

Sometimes the following line is needed to avoid an error::  
`processed_profile.dataset["time"] = ("id_dim", pd.to_datetime(processed_profile.dataset.time.values))`

Write processed profiles to file.

In [ ]:
processed_profile.dataset.to_netcdf(fn_out)